# ELAS — Notebook (0→7) avec données BAO/CMB intégrées

- **H₀ = 73** (cohérent SH0ES/Pantheon+)
- Distances intégrées **0 → z** (pas d'infini)
- **SN** : χ² avec **marginalisation de M**
- **BAO/CMB** : jeux de données **intégrés** (générés automatiquement), mais tu peux téléverser les tiens
- Sorties : Δχ², AIC, BIC + exports CSV

## 0) Préparation : imports + génération autom. des fichiers BAO/CMB

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# Génére les CSV BAO/CMB locaux (si tu ne téléverses rien, ceux-ci seront utilisés)
bao_df = pd.DataFrame([
    [0.106, "rs_over_DV", 0.336, 0.015],
    [0.15,  "rs_over_DV", 0.2239, 0.0084],
    [0.32,  "DM_over_rd", 8.47,   0.17],
    [0.57,  "DM_over_rd", 13.77,  0.13],
    [0.57,  "DH_over_rd", 9.00,   0.30],
    [0.70,  "DM_over_rd", 17.86,  0.31],
    [1.52,  "DM_over_rd", 30.21,  0.79],
    [2.33,  "DM_over_rd", 37.77,  2.13],
], columns=["z_eff","type","value","sigma"])
bao_df.to_csv("bao_data_ELAS.csv", index=False)

means_df = pd.DataFrame({
    "parameter": ["R","l_a","omega_b_h2"],
    "mean": [1.74963, 301.80845, 0.02237]
})
means_df.to_csv("means.csv", index=False)

inv_cov_df = pd.DataFrame({
    "R": [24434.0, -1166.4, -134214.0],
    "l_a": [-1166.4, 36.6, 543.2],
    "omega_b_h2": [-134214.0, 543.2, 64357310.0]
}, index=["R","l_a","omega_b_h2"])
inv_cov_df.to_csv("inv_cov.csv")

print("Fichiers écrits : bao_data_ELAS.csv, means.csv, inv_cov.csv")

Fichiers écrits : bao_data_ELAS.csv, means.csv, inv_cov.csv


## 1) Paramètres, modèles (ΛCDM / ELAS) et distances (0→z)

In [2]:
c = 299792.458   # km/s
H0 = 73.0        # calibration SH0ES
Omega_m = 0.315
Omega_b_h2 = 0.02237

ELAS_DEFAULT = dict(delta=0.05, Omega_osc=1.5, phi=-2.62)

try:
    from scipy.integrate import cumulative_trapezoid
except Exception:
    def cumulative_trapezoid(y, x, initial=0.0):
        y = np.asarray(y, dtype=float); x = np.asarray(x, dtype=float)
        trap = 0.5*(y[1:]+y[:-1])*(x[1:]-x[:-1])
        return np.concatenate(([initial], np.cumsum(trap)))

def E_LCDM(z, H0=H0, Omega_m=Omega_m):
    z = np.asarray(z, dtype=float)
    return np.sqrt(Omega_m*(1+z)**3 + (1.0 - Omega_m))

def E_ELAS(z, H0=H0, Omega_m=Omega_m, delta=0.05, Omega_osc=1.5, phi=-2.62):
    z = np.asarray(z, dtype=float)
    return np.sqrt(Omega_m*(1+z)**3 + (1.0 - Omega_m)*(1.0 + delta*np.cos(Omega_osc*np.log(1+z) + phi)))

def distances(z, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    z = np.asarray(z, dtype=float)
    zmax = float(np.max(z))
    zz = np.linspace(0.0, max(zmax, 1e-6), 6000)
    if model.upper()=="LCDM":
        EE = E_LCDM(zz, H0, Omega_m)
    else:
        pars = dict(ELAS_DEFAULT); pars.update(elas)
        EE = E_ELAS(zz, H0, Omega_m, **pars)
    Dc_grid = (c/H0) * cumulative_trapezoid(1.0/EE, zz, initial=0.0)
    Dc = np.interp(z, zz, Dc_grid)
    Dl = (1.0 + z) * Dc
    return dict(Dc=Dc, Dl=Dl)

def mu_theory(z, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    Dl = distances(z, model, H0, Omega_m, **elas)["Dl"]
    Dl = np.clip(Dl, 1e-6, None)
    return 5.0*np.log10(Dl) + 25.0

## 2) χ² SN avec marginalisation analytique de M

In [3]:
def chi2_sn_M_marginalized(z, mu_obs, sigma_mu, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    z = np.asarray(z, dtype=float)
    mu_obs = np.asarray(mu_obs, dtype=float)
    sigma_mu = np.asarray(sigma_mu, dtype=float)
    mu_th = mu_theory(z, model, H0, Omega_m, **elas)
    w = 1.0 / (sigma_mu**2)
    deltaM = np.sum(w*(mu_obs - mu_th)) / np.sum(w)
    resid = mu_obs - (mu_th + deltaM)
    chi2 = np.sum(w * resid**2)
    return float(chi2), float(deltaM)

## 3) Chargement des SN (upload ou mini-jeu de test intégré)

In [4]:
import pandas as pd
try:
    from google.colab import files
    print("➡️ Téléverse Pantheon CSV (colonnes: z,mu,sigma_mu). Sinon un mini-jeu sera utilisé.")
    up_sn = files.upload()
    sn_csv = None
    for k in up_sn.keys():
        if k.lower().endswith(".csv"):
            sn_csv = k; break
except Exception:
    sn_csv = None

if sn_csv is None:
    df_sn = pd.DataFrame({
        "z":[0.01,0.02,0.03,0.04,0.05,0.07,0.10,0.12,0.15,0.18,
             0.22,0.25,0.30,0.35,0.40,0.50,0.60,0.80,1.00,1.20],
        "mu":[33.2,34.1,35.0,35.7,36.4,37.5,38.4,38.9,39.6,40.2,
              40.9,41.3,41.9,42.4,42.8,43.5,44.0,44.7,45.2,45.6],
        "sigma_mu":[0.12]*20
    })
else:
    df_sn = pd.read_csv(sn_csv)

print("SN head:"); display(df_sn.head())
z_sn = df_sn["z"].values; mu_obs = df_sn["mu"].values; s_mu = df_sn["sigma_mu"].values

chi2_sn_LCDM, dM_LCDM = chi2_sn_M_marginalized(z_sn, mu_obs, s_mu, model="LCDM")
chi2_sn_ELAS, dM_ELAS = chi2_sn_M_marginalized(z_sn, mu_obs, s_mu, model="ELAS", **ELAS_DEFAULT)
print("SN χ² (LCDM, ELAS) =", chi2_sn_LCDM, chi2_sn_ELAS)

➡️ Téléverse Pantheon CSV (colonnes: z,mu,sigma_mu). Sinon un mini-jeu sera utilisé.


Saving pantheon_mini.csv to pantheon_mini.csv
SN head:


,z,mu,sigma_mu
0,0.01,33.2,0.12
1,0.02,34.1,0.12
2,0.03,35.0,0.12
3,0.04,35.7,0.12
4,0.05,36.4,0.12


SN χ² (LCDM, ELAS) = 577.7798913617594 587.521811530621


## 4) BAO : chargement (utilise **bao_data_ELAS.csv** intégré si rien n'est téléversé)

In [5]:
try:
    from google.colab import files
    print("➡️ (Optionnel) Téléverse BAO CSV (z_eff,type,value,sigma). Sinon bao_data_ELAS.csv intégré sera utilisé.")
    up_bao = files.upload()
    bao_csv = None
    for k in up_bao.keys():
        if k.lower().endswith(".csv"): bao_csv = k; break
except Exception:
    bao_csv = None

if bao_csv is None:
    bao_csv = "bao_data_ELAS.csv"  # intégré
df_bao = pd.read_csv(bao_csv)
print("BAO head:"); display(df_bao.head())

def rd_fid(): return 147.09

def bao_theory_value(z, typ, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    d = distances(np.array([z]), model, H0, Omega_m, **elas)
    Dm = d["Dc"][0]
    if model.upper()=="LCDM":
        Dh = (c/H0)/E_LCDM(np.array([z]), H0, Omega_m)[0]
    else:
        Dh = (c/H0)/E_ELAS(np.array([z]), H0, Omega_m, **elas)[0]
    rd = rd_fid()
    if typ=="DM_over_rd": return Dm/rd
    if typ=="DH_over_rd": return Dh/rd
    if typ=="DV_over_rd": return (z*Dh*Dm*Dm)**(1/3.0)/rd
    if typ=="rs_over_DV": return rd/((z*Dh*Dm*Dm)**(1/3.0))
    raise ValueError("type BAO inconnu")

def chi2_bao(df, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    if df is None or len(df)==0: return 0.0
    s = 0.0
    for _, r in df.iterrows():
        z, val, sig, typ = float(r["z_eff"]), float(r["value"]), float(r["sigma"]), str(r["type"])
        th = bao_theory_value(z, typ, model, H0, Omega_m, **elas)
        s += ((th - val)/sig)**2
    return float(s)

chi2_bao_LCDM = chi2_bao(df_bao, "LCDM")
chi2_bao_ELAS = chi2_bao(df_bao, "ELAS", **ELAS_DEFAULT)
print("BAO χ² (LCDM, ELAS) =", chi2_bao_LCDM, chi2_bao_ELAS)

➡️ (Optionnel) Téléverse BAO CSV (z_eff,type,value,sigma). Sinon bao_data_ELAS.csv intégré sera utilisé.


Saving bao_data_ELAS.csv to bao_data_ELAS (1).csv
BAO head:


,z_eff,type,value,sigma
0,0.106,rs_over_DV,0.3360,0.0150
1,0.150,rs_over_DV,0.2239,0.0084
2,0.320,DM_over_rd,8.4700,0.1700
3,0.570,DM_over_rd,13.7700,0.1300
4,0.570,DH_over_rd,9.0000,0.3000


BAO χ² (LCDM, ELAS) = 1449.571629722827 1457.192603931714


## 5) CMB : distance priors (utilise **means.csv** + **inv_cov.csv** intégrés si rien n'est téléversé)

In [9]:
try:
    from google.colab import files
    print("➡️ (Optionnel) Téléverse means.csv et inv_cov.csv. Sinon les fichiers intégrés seront utilisés.")
    up_cmb = files.upload()
    means_csv = invcov_csv = None
    for k in up_cmb.keys():
        if "means" in k and k.endswith(".csv"): means_csv = k
        if "inv_cov" in k and k.endswith(".csv"): invcov_csv = k
except Exception:
    means_csv = invcov_csv = None

import pandas as pd, numpy as np

if means_csv is None: means_csv = "means.csv"
if invcov_csv is None: invcov_csv = "inv_cov.csv"
cmb_means = pd.read_csv(means_csv)
cmb_icov  = pd.read_csv(invcov_csv, index_col=0)

def cmb_distance_priors_theory(H0=H0, Omega_m=Omega_m, Omega_b_h2=Omega_b_h2):
    zstar = 1089.0
    zgrid = np.linspace(0, zstar, 6000)
    Dc = (c/H0)*np.trapz(1.0/E_LCDM(zgrid, H0, Omega_m), zgrid)
    rs = 147.09
    R  = np.sqrt(Omega_m)*(H0/100.0) * (Dc/(c/100.0))
    la = np.pi * (Dc/rs)
    return float(R), float(la), float(Omega_b_h2)

def chi2_cmb_from_uploaded(means_df, icov_df, H0=H0, Omega_m=Omega_m):
    order = ["R","l_a","omega_b_h2"]
    vec_th = np.array(cmb_distance_priors_theory(H0, Omega_m, Omega_b_h2))
    vec_obs = means_df.set_index("parameter").loc[order,"mean"].values
    icov = icov_df.loc[order, order].values
    d = vec_th - vec_obs
    return float(d @ icov @ d)

chi2_cmb_LCDM = chi2_cmb_from_uploaded(cmb_means, cmb_icov, H0, Omega_m)
chi2_cmb_ELAS = chi2_cmb_from_uploaded(cmb_means, cmb_icov, H0, Omega_m)  # même proxy pour ce test
print("CMB χ² (LCDM, ELAS) =", chi2_cmb_LCDM, chi2_cmb_ELAS)

➡️ (Optionnel) Téléverse means.csv et inv_cov.csv. Sinon les fichiers intégrés seront utilisés.


Saving inv_cov.csv to inv_cov (2).csv
Saving means.csv to means (1).csv
CMB χ² (LCDM, ELAS) = 27165.218816685494 27165.218816685494


/tmp/ipython-input-999058264.py:22: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  Dc = (c/H0)*np.trapz(1.0/E_LCDM(zgrid, H0, Omega_m), zgrid)
/tmp/ipython-input-999058264.py:22: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  Dc = (c/H0)*np.trapz(1.0/E_LCDM(zgrid, H0, Omega_m), zgrid)


## 6) Totaux + Δχ², AIC, BIC + export

In [12]:
tot_LCDM = chi2_sn_LCDM + chi2_bao_LCDM + chi2_cmb_LCDM
tot_ELAS = chi2_sn_ELAS + chi2_bao_ELAS + chi2_cmb_ELAS

k_LCDM, k_ELAS = 0, 3
N_eff = len(df_sn) + len(df_bao) + 3  # 3 pour (R,l_a,omega_b_h2)

dchi2 = tot_LCDM - tot_ELAS
dAIC  = dchi2 - 2*(k_ELAS - k_LCDM)
dBIC  = dchi2 - (k_ELAS - k_LCDM)*np.log(max(N_eff,1))

summary = pd.DataFrame([
    dict(model="LCDM", H0=H0, Omega_m=Omega_m, chi2_sn=chi2_sn_LCDM, chi2_bao=chi2_bao_LCDM, chi2_cmb=chi2_cmb_LCDM, chi2_total=tot_LCDM),
    dict(model="ELAS", H0=H0, Omega_m=Omega_m, delta=ELAS_DEFAULT["delta"], Omega_osc=ELAS_DEFAULT["Omega_osc"], phi=ELAS_DEFAULT["phi"],
         chi2_sn=chi2_sn_ELAS, chi2_bao=chi2_bao_ELAS, chi2_cmb=chi2_cmb_ELAS, chi2_total=tot_ELAS)
])
print(summary)
print(f"\nΔχ² (LCDM − ELAS) = {dchi2}")
print(f"ΔAIC = {dAIC}  ΔBIC = {dBIC}   (N_eff = {N_eff} )")
summary.to_csv("ELAS_recalc_summary.csv", index=False)
print("➡️ Exports : ELAS_recalc_summary.csv")

  model    H0  Omega_m     chi2_sn     chi2_bao      chi2_cmb    chi2_total  \
0  LCDM  73.0    0.315  577.779891  1449.571630  27165.218817  29192.570338   
1  ELAS  73.0    0.315  587.521812  1457.192604  27165.218817  29209.933232   

   delta  Omega_osc   phi  
0    NaN        NaN   NaN  
1   0.05        1.5 -2.62  

Δχ² (LCDM − ELAS) = -17.36289437774758
ΔAIC = -23.36289437774758  ΔBIC = -27.66485599120302   (N_eff = 31 )
➡️ Exports : ELAS_recalc_summary.csv


## 7) Scan contraint (SN-only) : δ ≤ 0.15, 0.5 ≤ Ω ≤ 2.5

In [11]:
grid_delta = np.linspace(0.0, 0.15, 16)
grid_Omega = np.linspace(0.5, 2.5, 21)
grid_phi   = np.linspace(-np.pi, np.pi, 73)

best = dict(chi2=np.inf, delta=None, Omega=None, phi=None)
for d in grid_delta:
    for Om in grid_Omega:
        chi2_min = np.inf; phi_min = None
        for ph in grid_phi:
            chi2, _ = chi2_sn_M_marginalized(z_sn, mu_obs, s_mu,
                                             model="ELAS",
                                             delta=d, Omega_osc=Om, phi=ph)
            if chi2 < chi2_min:
                chi2_min, phi_min = chi2, ph
        if chi2_min < best["chi2"]:
            best.update(dict(chi2=chi2_min, delta=d, Omega=Om, phi=phi_min))

dchi2_best = chi2_sn_LCDM - best["chi2"]
print("Meilleur ELAS (scan contraint, SN-only):", best)
print(f"Δχ² (LCDM − ELAS_best) = {dchi2_best:.3f}")
pd.DataFrame([best]).to_csv("ELAS_best_SNonly_constrained.csv", index=False)
print("➡️ Export : ELAS_best_SNonly_constrained.csv")

Meilleur ELAS (scan contraint, SN-only): {'chi2': 540.9407616010558, 'delta': np.float64(0.15), 'Omega': np.float64(2.5), 'phi': np.float64(0.6981317007977319)}
Δχ² (LCDM − ELAS_best) = 36.839
➡️ Export : ELAS_best_SNonly_constrained.csv


In [13]:
# ======================================
# 📦 EXPORT GLOBAL ELAS (tous fichiers)
# ======================================

import shutil, os, zipfile, datetime

# Liste des fichiers générés à regrouper
exports = [
    "ELAS_recalc_summary.csv",
    "ELAS_best_SNonly_constrained.csv",
    "ELAS_delta_mu_H0_73.csv",
    "ELAS_delta_mu_H0_73.png",
    "bao_data_ELAS.csv",
    "means.csv",
    "inv_cov.csv"
]

# Dossier export
exp_dir = "ELAS_EXPORTS"
os.makedirs(exp_dir, exist_ok=True)

# Copie dans le dossier
for f in exports:
    if os.path.exists(f):
        shutil.copy(f, os.path.join(exp_dir, f))

# Création d’un fichier résumé
with open(os.path.join(exp_dir, "README_ELAS.txt"), "w") as f:
    f.write(f"ELAS exports regroupés — {datetime.datetime.now()}\n\n")
    f.write("Contenu :\n")
    for f2 in exports:
        if os.path.exists(os.path.join(exp_dir, f2)):
            f.write(f" - {f2}\n")

# Compression ZIP
zip_path = "ELAS_RESULTS_PACKAGE.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for root, _, files in os.walk(exp_dir):
        for name in files:
            path = os.path.join(root, name)
            zf.write(path, arcname=os.path.relpath(path, exp_dir))

print("✅ Export complet créé :", zip_path)
print("Contenu du dossier export :")
os.listdir(exp_dir)


✅ Export complet créé : ELAS_RESULTS_PACKAGE.zip
Contenu du dossier export :


['ELAS_recalc_summary.csv',
 'inv_cov.csv',
 'README_ELAS.txt',
 'means.csv',
 'ELAS_best_SNonly_constrained.csv',
 'bao_data_ELAS.csv']